# 决策树模型
一个树模型，每层的节点都是一个根据某个特征值来划分数据集的过程，最后达到叶子节点的节点才是最终的分类结果。

决策树的基本要素：
* 根节点
* 非子叶根节点
* 子叶节点

不同特征的划分能力不同，划分数据能力越强的放在越前的节点

衡量标准：熵值（随机变量不确定性的度量，说白了应该是混乱程度）

公式：
$$H(X)=-∑p(x)log_2p(x)$$





## 决策树的生成：

### ID3算法：基于信息增益的算法，是最常用的决策树生成算法。
数据集D的熵H(D)：
$$H(D)=∑_{i=1}^np(D_i)log_2p(D_i)$$
特征A对数据集D的条件熵H(D|A)：
$$H(D|A)=∑_{i=1}^np(D_i)H(D_i)$$
特征A对数据集D的信息增益：
$$g(D,A)=H(D)-H(D|A)$$
选择信息增益最大的特征作为分裂节点。

缺点：
* 如果某个特征值的类别很多，那么信息增益会很小，导致决策树过于平衡，无法准确分类。

### C4.5算法：是ID3算法的改进，相比ID3算法，C4.5算法在处理连续值变量时更加准确。
信息增益率：(表示特征X使得Y类别的信息的不确定性减少的程度)
$$g(D,A)=\frac{H(D)-H(D|A)}{H(D|A)}$$

### CART算法：是分类与回归树的统称，是一种二叉树，可以处理连续值和离散值变量。
Gini指数：(用这个替换熵值计算)
$$Gini(D)=∑_{k=1}^K(p_k(1-p_k))=1−∑_{k=1}^K(p_k)^2$$

## 处理连续值：
1. 先对特征值排序
2. 取两个相邻的特征值的平均值作为划分点
3. 计算这个划分点的信息增益/信息增益率/Gini指数
4. 选择信息增益/信息增益率/Gini指数最大的特征作为分裂节点

## 剪枝：
### 预剪枝：
建立决策树的过程中剪枝。

限制深度、叶子节点个数、节点的最小样本数、节点的最小信息增益。

### 后剪枝：
在生成决策树之后再剪枝。

通常会从决策树的叶节点开始，逐层向上对每个节点进行评估。如果剪掉该节点，带来的验证集中准确性差别不大或有明显提升，则可以对它进行剪枝，用叶子节点来代填该节点。

例如设置一个损失函数$C_\alpha(T) = C(T) + \alpha |T|$，其中$C(T)$是模型在训练集上的损失函数（比如信息增益），$\alpha$是正则化参数，$T$是当前节点的子树数量。


In [11]:
from matplotlib.font_manager import FontProperties  
import matplotlib.pyplot as plt  
from math import log  
import operator  
import pickle  

# 构造数据
def createDataSet():  
    dataSet = [          
        [0, 0, 0, 0, 'no'],  
        [0, 0, 0, 1, 'no'],  
        [0, 1, 0, 1, 'yes'],  
        [0, 1, 1, 0, 'yes'],  
        [0, 0, 0, 0, 'no'],  
        [1, 0, 0, 0, 'no'],  
        [1, 0, 0, 1, 'no'],  
        [1, 1, 1, 1, 'yes'],  
        [1, 0, 1, 2, 'yes'],  
        [1, 0, 1, 2, 'yes'], 
        [2, 0, 1, 2, 'yes'],  
        [2, 0, 1, 1, 'yes'],  
        [2, 1, 0, 1, 'yes'],
        [2, 1, 0, 2, 'yes'],
        [2, 0, 0, 0, 'no']  
    ]  
    labels = ['F1-AGE', 'F2-WORK', 'F3-HOME', 'F4-LOAN']  
    return dataSet, labels
# 递归建树
#featureLabels记录每层树划分的特征
def createTree(dataset, labels, featureLabels):
    # 检查当前节点是否纯了
    classlist = [example[-1] for example in dataset]
    if classlist.count(classlist[0])==len(classlist):
        return classlist[0]
    # 判断是否还剩下特征值可选来划分数据，每次选完了就把用的特征删掉
    if len(dataset[0]) == 1:
        return majoritCnt(classlist) # 返回最多的类别是什么
    bestFeature = chooseBestFeatureToSplit(dataset) # 选择最佳划分特征,这个返回的是下标
    bestFeatureLabel = labels[bestFeature]
    featureLabels.append(bestFeatureLabel)
    myTree = {bestFeatureLabel:{}} # 建立树的字典
    del labels[bestFeature] # 删除已经用过的特征
    featureValues = [example[bestFeature] for example in dataset]
    uniqueVals = set(featureValues) # 这里似乎没有考虑连续型数据
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatureLabel][value] = createTree(splitDataset(dataset, bestFeature, value), subLabels, featureLabels)
    return myTree
    
def majoritCnt(classlist):
    classCount = {} # 统计类别数量
    for vote in classlist:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1 
    sortedClassCount = sorted(classCount.items(),key=operator.itemgetter(1),reverse=True) # 排序
    #这里打个断点观察数据结构
    return sortedClassCount[0][0] # 返回最多的类别
    
def chooseBestFeatureToSplit(dataset):
    numFeatures = len(dataset[0]) - 1 # 最后一列是类别，所以特征数是numFeatures-1
    baseEntropy = calaShannonEnt(dataset) # 计算当前数据集的熵
    bestInfoGain = 0 # 记录信息增益最大值
    bestFeature = -1 # 记录信息增益最大值对应的特征

    for i in range(numFeatures):
        uniqueLabels = set(label[i] for label in dataset)
        newEntropy = 0
        InfoGain = 0
        for val in uniqueLabels:
            subDataset = splitDataset(dataset, i, val)    #分解数据集
            porp = len(subDataset)/len(dataset)# 你说的对但是元神是一款由米哈游独立研制的发放十
            newEntropy += porp * calaShannonEnt(subDataset)
        InfoGain = baseEntropy - newEntropy
        if bestInfoGain < InfoGain:
            bestInfoGain = InfoGain
            bestFeature = i
    # 返回选择的特征的下标值
    return bestFeature
    
# 熵值计算函数
def calaShannonEnt(dataset):
    numexamples = len(dataset)
    labelCount = {}      # 统计各个类别的数量
    classlist = [example[-1] for example in dataset]
    uniqueClass = set(classlist)
    for Class in uniqueClass:
        labelCount[Class]=classlist.count(Class)
    ShannonEnt = 0
    for key in labelCount:
        prop = float(labelCount[key])/numexamples
        ShannonEnt += -prop*log(prop,2)
    return ShannonEnt

def splitDataset(dataset,axis,val):
    subDataset=[]
    for FeatVec in dataset:
        if FeatVec[axis] == val:
            reduceFeatVec=FeatVec[:axis]
            reduceFeatVec.extend(FeatVec[axis+1:])
            subDataset.append(reduceFeatVec)
    return subDataset

if __name__=='__main__':
    Dataset,labels = createDataSet()
    featureLabels = []
    myTree = createTree(Dataset, labels, featureLabels)
    print(myTree)
    

{'F3-HOME': {0: {'F2-WORK': {0: 'no', 1: 'yes'}}, 1: 'yes'}}


下面是用鸢尾花数据集做一个sklearn的案例，并做一个可视化处理

In [3]:
from sklearn.datasets import load_iris  
from sklearn.tree import DecisionTreeClassifier  

iris = load_iris()  
X = iris.data[:, 2:]  # petal length and width  
y = iris.target  
# display(iris)

tree_clf = DecisionTreeClassifier(max_depth=2)  
tree_clf.fit(X, y)

DecisionTreeClassifier(max_depth=2)

In [6]:
# 得到每种结果的概率值，或者直接得到结果
display(
    tree_clf.predict_proba([[5,1.5]]),
    tree_clf.predict([[5,1.5]])
)

array([[0.        , 0.90740741, 0.09259259]])

array([1])

## 决策树的正则化
* min_samples_split(节点在分割之前必须含有的最小样本数)
* min_samples_leaf(叶子结点必须具有的最小样本数)
* max_leaf_nodes(叶子节点的最大数量)
* max_features(每个节点用来评估划分的最大特征数)(通常情况下不作限制)
* max_depth(最大深度)(一般控制的参数)

## 决策树回归
把Gini系数换成军方误差